In [ ]:
import requests
from bs4 import BeautifulSoup
from coupang_selenium_class import SearchManager as SM

class shopping():       
        def __init__(self, driver):
                self.driver = driver

        def request(self):
                raw = SM(self.driver).page_source()
                search = BeautifulSoup(raw, 'html.parser')
                return search
        
        def find_html(self):
                product_num = 1
                
                link_list, imageLink_list, title_list, price_list, point_list=[], [], [], [], [] 

                search = self.request()

                box = search.find('ul', {'id' : 'productList'})
                all_product = box.find_all('li', {'class': 'search-product'})

                for i, product in enumerate(all_product):
                        imagefind = product.find("img", {'class' : 'search-product-wrap-img'})
                        if imagefind.get("data-img-src") is None:
                                imageLink = "http:" + imagefind.get('src')
                        else:
                                imageLink = "http:" + imagefind.get("data-img-src")

                        title = product.find('div', {'class' : 'name'})
                        price = product.find('strong', {'class' : 'price-value'})
                        point = product.find('span', {'class' : 'rating-total-count'})
                        
                        if point is None:
                                point=''

                        link2 = product.find('a')['href']
                        link = "https://www.coupang.com" + link2

                        imageLink_list.append(imageLink)
                        
                        title_text = f"{product_num}. {title.text.strip()}"

                        title_list.append(title_text)

                        if isinstance(point, str):
                                price_list.append(price)
                        else:
                                price_list.append(price.text)
                        

                        if isinstance(point, str):
                                point_list.append(point)
                        else:
                                point_list.append(point.text)
                        
                        link_list.append(link)    

                        product_num += 1

                return link_list, imageLink_list, title_list, price_list, point_list
                

In [ ]:
import requests
from bs4 import BeautifulSoup
from Class_2_Searching_coupang import CoupangManager as CM

class ShopManager():       
        def __init__(self, driver):
                self.driver = driver

        def request(self):
                raw = CM(self.driver).page_source()
                search = BeautifulSoup(raw, 'html.parser')
                return search

        def move_page(self, page_num):
                ## 페이지가 1일때
                url = self.driver.current_url()
                
                if "page=" in url: 
                        url = url.replace("page=1", f"page={page_num}")
                else:
                        url += f"&page={page_num}"

                header={'User-Agent': ( "Mozilla/5.0 (Windows NT 10.0; Win64; x64) " 
                                "AppleWebKit/537.36 (KHTML, like Gecko) " 
                                "Chrome/121.0.0.0 Safari/537.36"), 
                                "Accept-Language": "ko-KR,ko;q=0.9"}
                
                raw = requests.get(url, headers=header)
                search = BeautifulSoup(raw.text, 'html.parser')
                return url, search
                
        def find_html(self, page_num):
                product_num = 1

                link_list, imageLink_list, title_list, price_list, point_list=[], [], [], [], [] 

                if page_num==1:
                        search = self.request()
                else:
                        search = self.move_page(page_num)

                box = search.find('ul', {'id' : 'productList'})
                all_product = box.find_all('li', {'class': 'search-product'})

                for i, product in enumerate(all_product):
                        imagefind = product.find("img", {'class' : 'search-product-wrap-img'})
                        if imagefind.get("data-img-src") is None:
                                imageLink = "http:" + imagefind.get('src')
                        else:
                                imageLink = "http:" + imagefind.get("data-img-src")

                        title = product.find('div', {'class' : 'name'})
                        price = product.find('strong', {'class' : 'price-value'})
                        point = product.find('span', {'class' : 'rating-total-count'})

                        link2 = product.find('a')['href']
                        link = "https://www.coupang.com" + link2

                        imageLink_list.append(imageLink)
                        
                        title_text = f"{product_num}. {title.text.strip()}"

                        title_list.append(title_text)

                        if price is None:
                                price_list.append('')
                        else:
                                price_list.append(price.text)
                        

                        if point is None:
                                point_list.append('')
                        else:
                                point_list.append(point.text)
                        
                        link_list.append(link)    

                        product_num += 1

                return link_list, imageLink_list, title_list, price_list, point_list


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import subprocess
import chromedriver_autoinstaller
import re

try:
    chrome_path = (
        r'C:\Program Files\Google\Chrome\Application\chrome.exe '
        r'--remote-debugging-port=9221 --user-data-dir="C:\chrometemp" '
        r'--window-size=400,300 --window-position=0,0'
    )
except:
    chrome_path = (
        r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe '
        r'--remote-debugging-port=9221 --user-data-dir="C:\chrometemp" '
        r'--window-size=400,300 --window-position=0,0'
    )
finally:
    chrome_process = subprocess.Popen(chrome_path)

options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9221")

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=options)
driver.get("https://www.coupang.com/")

elem = driver.find_element(By.ID, 'headerSearchKeyword')
elem.send_keys('노트북')
elem.send_keys("\n")

url = driver.current_url

page_num=2

if re.search(r'page=\d+', url):
    new_url = re.sub(r'page=\d+', f'page={page_num}', url)
else:
    separator = '&' if '?' in url else '?'
    new_url = f'{url}{separator}page={page_num}'

raw = driver.page_source

In [12]:
new_url

'https://www.coupang.com/np/search?component=&q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&page=2'

In [5]:
chrome_process.terminate()